In [1]:
%config IPCompleter.use_jedi = False

In [2]:
import cv2
import tensorflow as tf
from keras.models import load_model
import numpy as np
import mediapipe as mp

2021-08-31 21:42:03.527937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 21:42:03.527978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
model = load_model("final_model.h5")

2021-08-31 21:42:05.246657: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-31 21:42:05.246697: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-31 21:42:05.246717: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Rupanshu): /proc/driver/nvidia/version does not exist
2021-08-31 21:42:05.246929: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def prob_viz(input_frame, prob, prev_prob, curr_count):
    output_frame = input_frame.copy()
    
    num = 1
    
    count = curr_count
    
#     cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), [0, 255, 0], -1)
    if prob > 0.95 and prev_prob is not None and prev_prob <= 0.95:
        count += 1
    cv2.putText(output_frame, f"SQUAT count: {count}", (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame, count

In [6]:
stream = cv2.VideoCapture(0)

In [7]:
def get_reqd_landmarks(pose):
    try:
        pose_landmarks = [[res.x, res.y] for res in pose.pose_landmarks.landmark]
        reqd_landmarks = np.array(pose_landmarks)[[11, 12, 23, 24, 25, 26]]
        return reqd_landmarks
    except Exception as e:
        return None

In [8]:
sequence = []
prev_prob = None
curr_count = 0

with mp_pose.Pose(min_detection_confidence=0.2, min_tracking_confidence=0.2) as pose:
    while stream.isOpened():
        ret, frame = stream.read()

        if ret is False:
            break

        extracted_pose = pose.process(frame)
        if extracted_pose is None:
            continue
            
        landmarks = get_reqd_landmarks(extracted_pose)
        
        if landmarks is None:
            continue
        
        landmarks = landmarks.ravel().tolist()
                
        sequence.append(landmarks)
        
        sequence = sequence[-75:]
                
        if len(sequence) == 75:
            prob = model.predict(np.expand_dims(np.array(sequence), axis=0)).item()
            
            op_frame, curr_count = prob_viz(frame, prob, prev_prob, curr_count)
            
            cv2.imshow('Webcam', op_frame)
            
            prev_prob = prob

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                cv2.destroyAllWindows()
                
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2021-08-31 21:42:19.487878: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
